In [1]:
import os
import json
import numpy as np
from tqdm import tqdm_notebook

magazine_path = '/data/private/Arena/datasets/magazine.json'
metadata_path = '/data/private/Arena/datasets/metadata.json'
users_path = '/data/private/Arena/datasets/users.json'
predict_path = '/data/private/Arena/datasets/predict/'
read_path = '/data/private/Arena/datasets/read/'

In [2]:
#magazine -> keyword list
magazine_list = []
for line in open(magazine_path, 'r', encoding='utf-8'):
    magazine_list.append(json.loads(line))
metadata_list = []
for line in open(metadata_path, 'r', encoding='utf-8'):
    metadata_list.append(json.loads(line))
users_list = []
for line in open(users_path, 'r', encoding='utf-8'):
    users_list.append(json.loads(line))

In [3]:
magazine_list[0], metadata_list[0], users_list[0]

({'magazine_tag_list': ['브런치북', '육아일기', '대화법', '들려주고픈이야기'], 'id': '38842'},
 {'magazine_id': 8982,
  'user_id': '@bookdb',
  'title': '사진으로 옮기기에도 아까운, 리치필드 국립공원',
  'keyword_list': ['여행', '호주', '국립공원'],
  'display_url': 'https://brunch.co.kr/@bookdb/782',
  'sub_title': '세상 어디에도 없는 호주 Top 10',
  'reg_ts': 1474944427000,
  'article_id': 782,
  'id': '@bookdb_782'},
 {'keyword_list': [],
  'following_list': ['@perytail', '@brunch'],
  'id': '#901985d8bc4c481805c4a4f911814c4a'})

## keyword / reader / writer dict and list

In [ ]:
keyword_list = ['unk', 'pad']
for data in tqdm_notebook(magazine_list):
    for keyword in data['magazine_tag_list']:
        if keyword not in keyword_list:
            keyword_list.append(keyword)
for data in tqdm_notebook(metadata_list):
    for keyword in data['keyword_list']:
        if keyword not in keyword_list:
            keyword_list.append(keyword)
for data in tqdm_notebook(users_list):
    for keyword in data['keyword_list']:
        if keyword not in keyword_list:
            keyword_list.append(keyword)
            
keyword_dict = {}
for i, keyword in enumerate(keyword_list):
    keyword_dict[keyword] = i
    
np.save('/data/private/Arena/prepro_results/keyword_dict.npy', keyword_dict)
np.save('/data/private/Arena/prepro_results/keyword_list.npy', keyword_list)

In [60]:
id2writer = ['unk']
id2reader = ['unk']
for data in tqdm_notebook(users_list):
    id2reader.append(data['id'])
    for writer in data['following_list']:
        if writer not in id2writer:
            id2writer.append(writer)

reader2id = {}
for i, reader in enumerate(id2reader):
    reader2id[reader] = i
    
writer2id = {}
for i, writer in enumerate(id2writer):
    writer2id[writer] = i
    
np.save('/data/private/Arena/prepro_results/id2reader.npy', id2reader)
np.save('/data/private/Arena/prepro_results/reader2id.npy', reader2id)
np.save('/data/private/Arena/prepro_results/id2writer.npy', id2writer)
np.save('/data/private/Arena/prepro_results/writer2id.npy', writer2id)

In [26]:
magazine2id = {'unk':0}
id2magazine = ['unk']
for data in tqdm_notebook(magazine_list):
    id_ = int(data['id'])
    
    if id_ not in id2magazine:
        magazine2id[id_] = len(id2magazine)
        id2magazine.append(id_)
        
np.save('/data/private/Arena/prepro_results/id2magazine.npy', id2magazine)
np.save('/data/private/Arena/prepro_results/magazine2id.npy', magazine2id)

## reader / item 2 elements & item dict and list

In [129]:
reader2elem = {'unk':0}
follow_list = []
follow_maxlen = 12
keyword_maxlen = 0
for data in tqdm_notebook(users_list):
    #follow_list.append(len(data['following_list']))
    id_ = reader2id[data['id']]
    follow = [writer2id['unk']] * (follow_maxlen-len(data['following_list'])) + \
             list(map(writer2id.get, data['following_list'][-follow_maxlen:]))
    reader2elem[id_] = follow

    #reader2elem[id_] = 
    #if keyword_maxlen < len(data['keyword_list']):
    #    keyword_maxlen = len(data['keyword_list'])
    #if data['keyword_list']:
    #    keywords = [kw['keyword'].split(' ') for kw in data['keyword_list'] if kw['cnt'] > 10]
    #    keywords = list(set([a for b in keywords for a in b]))
    #    print(keywords)
    #    if keyword_maxlen < len(keywords):
    #        keyword_maxlen = len(keywords)

np.save('/data/private/Arena/prepro_results/reader2elem.npy', reader2elem)

In [56]:
keywordlen_list = []
for data in tqdm_notebook(users_list):
    keywordlen_list.append(len(data['keyword_list']))

In [67]:
np.sort(keywordlen_list)[280000], len(keywordlen_list)

(0, 310758)

In [120]:
print(np.mean(follow_list))
print(sorted(follow_list)[int(len(follow_list)*0.85)])

8.420954569150272
11


In [29]:
keywd_maxlen = 5
item2elem = {0:[0,0,0,0,0,0,0,0]}
item_list = ['unk']
item_dict = {'unk':0}

for data in tqdm_notebook(metadata_list):
    if item_dict.get(data['id']) == None:
        item_dict[data['id']] = len(item_list)
        item_list.append(data['id'])
        
    if data['keyword_list']:
        keywd = [keyword_dict['pad']] * (keywd_maxlen-len(data['keyword_list'])) + \
                list(map(keyword_dict.get, data['keyword_list'][::-1]))
    else:
        keywd = [keyword_dict['unk']] * keywd_maxlen
    writer = writer2id[data['user_id']]
    reg_ts = int(data['reg_ts'])
    if magazine2id.get(int(data['magazine_id'])) == None:
        magazine2id[int(data['magazine_id'])] = len(id2magazine)
        id2magazine.append(int(data['magazine_id']))
    mag_id = magazine2id[int(data['magazine_id'])]
    
    item2elem[item_dict[data['id']]] = [writer] + keywd + [reg_ts, mag_id]
        
np.save('/data/private/Arena/prepro_results/item_dict.npy', item_dict)
np.save('/data/private/Arena/prepro_results/item_list.npy', item_list)
np.save('/data/private/Arena/prepro_results/item2elem.npy', item2elem)
np.save('/data/private/Arena/prepro_results/id2magazine.npy', id2magazine)
np.save('/data/private/Arena/prepro_results/magazine2id.npy', magazine2id)

## Valid Tensor & Writer 2 items (X)

In [142]:
import numpy as np
from tqdm import tqdm_notebook
import torch

id2writer = np.load('/data/private/Arena/prepro_results/id2writer.npy')
writer2id = np.load('/data/private/Arena/prepro_results/writer2id.npy', allow_pickle=True).item()
keyword_dict = np.load('/data/private/Arena/prepro_results/keyword_dict.npy', allow_pickle=True).item()
keyword_list = np.load('/data/private/Arena/prepro_results/keyword_list.npy')
valid_writer_keywd = []


for data in tqdm_notebook(metadata_list):
    if data['keyword_list']:
        keywd = [keyword_dict['pad']] * (keywd_maxlen-len(data['keyword_list'])) + \
                list(map(keyword_dict.get, data['keyword_list'][::-1]))
    else:
        keywd = [keyword_dict['unk']] * keywd_maxlen
    writer = writer2id[data['user_id']]
    reg_ts = int(data['reg_ts'])
    meg_id = int(data['magazine_id'])
    item_id = item_dict[data['id']]
    
    valid_writer_keywd.append([item_id, writer] + keywd + [reg_ts, meg_id])

valid_writer_keywd = torch.from_numpy(np.array(valid_writer_keywd))
torch.save(valid_writer_keywd, '/data/private/Arena/prepro_results/valid_writer_keywd.pkl')

In [27]:
item_dict = np.load('/data/private/Arena/prepro_results/item_dict.npy', allow_pickle=True).item()
item_list = np.load('/data/private/Arena/prepro_results/item_list.npy')

writerid2items = {}
for data in tqdm_notebook(metadata_list):
    user_id = writer2id[data['user_id']]
    id_ = item_dict[data['id']]
    keyword = keyword_dict[data['keyword_list'][0] if data['keyword_list'] is True else '없음']
    
    if writerid2items.get(user_id) == None:
        writerid2items[user_id] = [[id_, user_id, keyword]]
    else:
        writerid2items[user_id].append([id_, user_id, keyword])
        
np.save('/data/private/Arena/prepro_results/writerid2items.npy', writerid2items)


## From 2019022 : Reader 2 Read item

In [81]:
file_list = os.listdir(read_path)
file_list.sort()

In [89]:
file_list[3456:]

['2019022200_2019022201',
 '2019022201_2019022202',
 '2019022202_2019022203',
 '2019022203_2019022204',
 '2019022204_2019022205',
 '2019022205_2019022206',
 '2019022206_2019022207',
 '2019022207_2019022208',
 '2019022208_2019022209',
 '2019022209_2019022210',
 '2019022210_2019022211',
 '2019022211_2019022212',
 '2019022212_2019022213',
 '2019022213_2019022214',
 '2019022214_2019022215',
 '2019022215_2019022216',
 '2019022216_2019022217',
 '2019022217_2019022218',
 '2019022218_2019022219',
 '2019022219_2019022220',
 '2019022220_2019022221',
 '2019022221_2019022222',
 '2019022222_2019022223',
 '2019022223_2019022300',
 '2019022300_2019022301',
 '2019022301_2019022302',
 '2019022302_2019022303',
 '2019022303_2019022304',
 '2019022304_2019022305',
 '2019022305_2019022306',
 '2019022306_2019022307',
 '2019022307_2019022308',
 '2019022308_2019022309',
 '2019022309_2019022310',
 '2019022310_2019022311',
 '2019022311_2019022312',
 '2019022312_2019022313',
 '2019022313_2019022314',
 '2019022314

In [90]:
reader2item = {}
#file_list = [x for x in os.listdir(read_path) if x.startswith('20190222')]

for read_file in tqdm_notebook(file_list[3456:]):
    try:
        file = open(read_path+read_file, 'r')
        data_ = file.readlines()
    except:
        print(read_file)
        continue

    for line in data_:
        tokens = line.split(' ')
        try:
            reader = reader2id[tokens[0]]
        except:
            continue
        items = [item_dict[x] if item_dict.get(x)!=None else item_dict['unk'] for x in tokens[1:-1]]
        
        if reader2item.get(reader) != None:
            reader2item[reader] = reader2item[reader] + items
        else:
            reader2item[reader] = items
            
#np.save('/data/private/Arena/prepro_results/reader2item.npy', reader2item)

In [79]:
len(reader2item), len(reader2id)

(115730, 310759)

In [73]:
userid2followid = {}
for data in users_list:
    id_ = reader2id[data['id']]
    following_list = [writer2id[x] for x in data['following_list']]
    userid2followid[id_] = following_list
    
np.save('/data/private/Arena/prepro_results/userid2followid.npy', userid2followid)

In [74]:
max_keylen = 0
for data in tqdm_notebook(metadata_list):
    keylen = len(data['keyword_list'])
    if max_keylen < keylen:
        max_keylen = keylen
print(max_keylen)

5


In [50]:
#reader2item = np.load('/data/private/Arena/prepro_results/reader2item.npy', allow_pickle=True).item()
readlen_list = [len(v) for v in reader2item.values()]

In [51]:
np.median(readlen_list)

6.0

# RNN Based
Train Valid Test data (from 4 to 1 : window size = 5) and Mask 

In [75]:
rnn_train_data = []
rnn_valid_data = []
rnn_test_data = {}
window_size = 5
for reader, items in reader2item.items():
    if len(items) < window_size:
        items = [item_dict['unk']] * (window_size-len(items)) + items
        
    rnn_data = []
    for i in range(len(items)-window_size+1):
        rnn_data.append([reader]+ items[i:i+window_size])

    if len(items) > window_size+4:
        rnn_train_data += rnn_data[:int(len(rnn_data)*0.9)]
        rnn_valid_data += rnn_data[int(len(rnn_data)*0.9):]
    else:
        rnn_train_data += rnn_data
    rnn_test_data[reader] = rnn_data[-1]
        
np.save('/data/private/Arena/prepro_results/rnn_train_data.npy', rnn_train_data)
np.save('/data/private/Arena/prepro_results/rnn_valid_data.npy', rnn_valid_data)
#np.save('/data/private/Arena/prepro_results/rnn_test_data.npy', rnn_test_data)

In [76]:
len(rnn_train_data), len(rnn_valid_data)

(4045338, 467044)

Test data as not a dictionary (REAL)

In [77]:
pred_dev_file = '/data/private/Arena/datasets/predict/dev.users'
pred_dev_data = open(pred_dev_file, 'r').readlines()
test_data = []
a = 0
for line in pred_dev_data:
    if reader2id.get(line.strip()) != None:
        reader = reader2id[line.strip()]
        readed = rnn_test_data[reader]
    else:
        reader = reader2id['unk']
        readed = [0,0,0,0,0,0]
        a += 1
    test_data.append(readed)

np.save('/data/private/Arena/prepro_results/rnn_test_data.npy', np.array(test_data))
print(a)

9


In [ ]:
pred_dev_file = '/data/private/Arena/datasets/predict/dev.users'
pred_dev_data = open(pred_dev_file, 'r').readlines()

reader2id = np.load('/data/private/Arena/prepro_results/reader2id.npy', allow_pickle=True).item()
reader2items = np.load('/data/private/Arena/prepro_results/reader2item.npy', allow_pickle=True).item()
dev_mask = np.ones((len(pred_dev_data), 643105))
for i, line in enumerate(pred_dev_data):
    try:
        readed = reader2items[reader2id[line.strip()]]
    except:
        continue
    dev_mask[i,readed] = 0
dev_mask[:,0] = 0
    
np.save('/data/private/Arena/prepro_results/dev_mask.npy', dev_mask)


Train Valid Test data and Mask (from 4 to 2 : window size = 6)

In [53]:
rnn_train_data = []
rnn_valid_data = []
rnn_test_data = {}
window_size = 6

for reader, items in reader2item.items():
    if len(items) < window_size:
        items = [item_dict['unk']] * (window_size-len(items)) + items
        rnn_test_data[reader] = [reader] + items
        continue
        
    rnn_data = []
    for i in range(len(items)-window_size+1):
        rnn_data.append([reader]+ items[i:i+window_size])

    if len(items) > window_size+4:
        rnn_train_data += rnn_data[:int(len(rnn_data)*0.9)]
        rnn_valid_data += rnn_data[int(len(rnn_data)*0.9):]
    else:
        rnn_train_data += rnn_data
        
    rnn_test_data[reader] = rnn_data[-1]

np.save('/data/private/Arena/prepro_results/rnn_train_data2.npy', rnn_train_data)
np.save('/data/private/Arena/prepro_results/rnn_valid_data2.npy', rnn_valid_data)

pred_dev_file = '/data/private/Arena/datasets/predict/dev.users'
pred_dev_data = open(pred_dev_file, 'r').readlines()
test_data = []
a = 0
for line in pred_dev_data:
    if reader2id.get(line.strip()) != None:
        reader = reader2id[line.strip()]
        readed = rnn_test_data[reader]
    else:
        reader = reader2id['unk']
        readed = [0] * (window_size+1)
        a += 1
    print(readed)
    test_data.append(readed)

np.save('/data/private/Arena/prepro_results/rnn_test_data2.npy', np.array(test_data))
print(a)

In [55]:
len(rnn_train_data), len(rnn_valid_data)

(1095568, 129972)